In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re


Getting stopwords to filter out meaningless data

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
news_df= pd.read_csv('train.csv')

In [14]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [15]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [21]:
news_df=news_df.fillna('')
news_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [22]:
news_df['news']=news_df['author']+' '+news_df['title']

In [23]:
news_df['news']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: news, Length: 20800, dtype: object

In [32]:
x = news_df[['id','title','text','news']]
y = news_df['label']

In [39]:
x.shape


(20800, 4)

In [40]:
y.shape

(20800,)

Stemming

In [50]:
port_stem = PorterStemmer()
def stemming(news):
    stm_news = re.sub('[^a-zA-Z]',' ',news)
    stm_news = stm_news.lower().split()
    stm_news = [port_stem.stem(word) for word in stm_news if not word in stopwords.words('english')]
    stm_news = ' '.join(stm_news)
    return stm_news

In [51]:
news_df['news'] = news_df['news'].apply(stemming)

In [55]:
x=news_df['news']
y=news_df['label']

In [56]:
#Text to tfidf numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [57]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, stratify=y, random_state=2)

Training


In [66]:
model=LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [75]:
x_train_prediction = model.predict(x_train)
train_accuracy = accuracy_score(x_train_prediction, y_train)

In [76]:
x_test_prediction=model.predict(x_test)
test_accuracy= accuracy_score(x_test_prediction, y_test)

In [77]:
print('Accuracy_train : ', train_accuracy*100)
print('Accuracy_test : ', test_accuracy*100)


Accuracy_train :  98.7160633484163
Accuracy_test :  97.8525641025641


Predict

In [86]:
x_new = x_test[8]

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('Real news!')
else:
  print('Fake news!')

[1]
Fake news!
